In [ ]:
import numpy as np
import heapq
import matplotlib.pyplot as plt

In [ ]:
def eikonal_solver(speed, grid_size, box_size):
    nx, ny = grid_size
    dx, dy = box_size[0] / (nx - 1), box_size[1] / (ny - 1)

    grid = np.full(grid_size, np.inf)
    grid[0, :] = 0.0
    grid[-1, :] = 0.0
    grid[:, 0] = 0.0
    grid[:, -1] = 0.0

    while np.isinf(grid).any():
        indices = np.argwhere(np.isinf(grid))
        min_idx = np.argmin(grid[tuple(indices.T)])
        idx = tuple(indices[min_idx])

        grid[idx] = compute_time(speed, dx, dy, idx, grid)

    return grid

def compute_time(speed, dx, dy, idx, grid):
    nx, ny = grid.shape
    i, j = idx
    neighbors = []

    if i > 0:
        neighbors.append((i - 1, j))
    if i < nx - 1:
        neighbors.append((i + 1, j))
    if j > 0:
        neighbors.append((i, j - 1))
    if j < ny - 1:
        neighbors.append((i, j + 1))

    times = []
    for neighbor in neighbors:
        ni, nj = neighbor
        dt = np.sqrt(dx**2 + dy**2) / speed[i, j] + grid[ni, nj]
        times.append(dt)

    return min(times)

speed = np.ones((50, 50))  
grid_size = (50, 50)     
box_size = (1.0, 1.0)    

solution = eikonal_solver(speed, grid_size, box_size)
print(solution)

In [ ]:
def visualize_solution(solution, box_size):
    nx, ny = solution.shape
    dx, dy = box_size[0] / (nx - 1), box_size[1] / (ny - 1)

    x = np.linspace(0, box_size[0], nx)
    y = np.linspace(0, box_size[1], ny)
    X, Y = np.meshgrid(x, y)

    plt.contourf(X, Y, solution.T, levels=20, cmap='jet')
    plt.colorbar(label='Travel Time')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Eikonal Equation Solution')
    plt.show()

speed = np.ones((50, 50))  
grid_size = (50, 50)       
box_size = (1.0, 1.0)      

solution = eikonal_solver(speed, grid_size, box_size)
visualize_solution(solution, box_size)

In [ ]:
# 8 stencils center

def solve_eikonal(size, start):
    distance = np.full((size, size), np.inf)
    visited = np.zeros((size, size), dtype=bool)
    heap = []

    distance[start] = 0
    heapq.heappush(heap, (0, start))

    while heap:
        (dist, (x, y)) = heapq.heappop(heap)

        if visited[x, y]:
            continue

        visited[x, y] = True
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            nx, ny = x + dx, y + dy

            if nx < 0 or nx >= size or ny < 0 or ny >= size:
                continue  

            new_dist = dist + np.sqrt(dx ** 2 + dy ** 2)  

            if new_dist < distance[nx, ny]:  
                distance[nx, ny] = new_dist
                heapq.heappush(heap, (new_dist, (nx, ny)))

    return distance

size = 100
start = (size//2, size//2) 
distance = solve_eikonal(size, start)

plt.imshow(distance, cmap='jet', interpolation='nearest')
plt.colorbar(label='Distance')
plt.title('Sound Propagation from the Center of the Box')
plt.show()


In [ ]:
#3d eikonal

FAR = np.inf
ACCEPTED = -1
CONSIDERED = -2

directions = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1),
              (1, 1, 0), (-1, -1, 0), (1, -1, 0), (-1, 1, 0), (1, 0, 1), (-1, 0, -1),
              (1, 0, -1), (-1, 0, 1), (0, 1, 1), (0, -1, -1), (0, 1, -1), (0, -1, 1),
              (1, 1, 1), (-1, -1, -1), (1, 1, -1), (-1, -1, 1), (1, -1, 1), (-1, 1, -1),
              (1, -1, -1), (-1, 1, 1)]


def solve_eikonal(size):
    # Initialize arrays
    u = np.full((size, size, size), FAR)
    status = np.full((size, size, size), FAR)
    heap = []

    source = (size//2, size//2, size//2)
    u[source] = 0
    status[source] = ACCEPTED
    for d in directions:
        neighbor = tuple(s + d[i] for i, s in enumerate(source))
        if 0 <= neighbor[0] < size and 0 <= neighbor[1] < size and 0 <= neighbor[2] < size:
            status[neighbor] = CONSIDERED
            heapq.heappush(heap, (u[source] + np.sqrt(sum(np.array(d)**2)), neighbor))

    while heap:
        (dist, (x, y, z)) = heapq.heappop(heap)
        
        if status[x, y, z] != CONSIDERED:
            continue

        status[x, y, z] = ACCEPTED

        for d in characteristic_direction((x, y, z)):
            nx, ny, nz = x + d[0], y + d[1], z + d[2]

            if nx < 0 or nx >= size or ny < 0 or ny >= size or nz < 0 or nz >= size:
                continue

            if status[nx, ny, nz] != ACCEPTED:
                new_dist = dist + np.sqrt(sum(np.array(d)**2)) 

                if new_dist < u[nx, ny, nz]:
                    u[nx, ny, nz] = new_dist
                    status[nx, ny, nz] = CONSIDERED
                    heapq.heappush(heap, (new_dist, (nx, ny, nz)))

    return u


def characteristic_direction(node):
  
    return directions

size = 50
u = solve_eikonal(size)

import matplotlib.pyplot as plt
plt.imshow(u[size//2, :, :], cmap='hot', interpolation='nearest')
plt.colorbar(label='u')
plt.title('Solution to Eikonal Equation')
plt.show()


In [ ]:
#8 stencils square


FAR = np.inf
ACCEPTED = -1
CONSIDERED = 1

directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (1, -1), (-1, 1)]

def solve_eikonal(size):
    u = np.full((size, size), FAR)
    status = np.full((size, size), FAR)
    heap = []

    for x in range(size):
        for y in range(size):
            if x == 0 or x == size - 1 or y == 0 or y == size - 1:
                u[x, y] = 0
                status[x, y] = ACCEPTED
                for dx, dy in directions:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < size and 0 <= ny < size and status[nx, ny] != ACCEPTED:
                        status[nx, ny] = CONSIDERED
                        dist = np.sqrt(dx ** 2 + dy ** 2)
                        heapq.heappush(heap, (dist, (nx, ny)))

    while heap:
        (dist, (x, y)) = heapq.heappop(heap)
        
        if status[x, y] != CONSIDERED:
            continue

        status[x, y] = ACCEPTED

        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            if nx < 0 or nx >= size or ny < 0 or ny >= size:
                continue

            if status[nx, ny] != ACCEPTED:
                new_dist = dist + np.sqrt(dx ** 2 + dy ** 2)  

                if new_dist < u[nx, ny]:
                    u[nx, ny] = new_dist
                    status[nx, ny] = CONSIDERED
                    heapq.heappush(heap, (new_dist, (nx, ny)))

    return u

size = 100
u = solve_eikonal(size)

plt.imshow(u, cmap='jet', interpolation='nearest')
plt.colorbar(label='Distance')
plt.title('Equiarrival Curves from All Borders of the Square')
plt.show()

level_set = 35

# Binary Image
binary_image = np.isclose(u, level_set, atol=0.5)
plt.imshow(binary_image, cmap='Greys', interpolation='nearest')
plt.title(f'Level set at {level_set}')
plt.show()


In [ ]:
#incorporating slowness

import numpy as np
import heapq
import matplotlib.pyplot as plt

FAR = np.inf
ACCEPTED = -1
CONSIDERED = -2

directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]

def solve_eikonal(size):
    u = np.full((size, size), FAR)
    status = np.full((size, size), FAR)
    heap = []

    # Speed Field 
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    speed = np.sqrt(x**2 + y**2)
    speed = np.where(speed > 0, 1/speed, FAR) 

    for x in range(size):
        for y in range(size):
            if x == 0 or x == size - 1 or y == 0 or y == size - 1:
                u[x, y] = 0
                status[x, y] = ACCEPTED
                for dx, dy in directions:
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < size and 0 <= ny < size and status[nx, ny] != ACCEPTED:
                        status[nx, ny] = CONSIDERED
                        heapq.heappush(heap, (np.sqrt(dx ** 2 + dy ** 2) / speed[nx, ny], (nx, ny)))

    while heap:
        (dist, (x, y)) = heapq.heappop(heap)

        if status[x, y] != CONSIDERED:
            continue

        status[x, y] = ACCEPTED

        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            if nx < 0 or nx >= size or ny < 0 or ny >= size:
                continue

            if status[nx, ny] != ACCEPTED:
                new_dist = dist + np.sqrt(dx ** 2 + dy ** 2) / speed[nx, ny]  

                if new_dist < u[nx, ny]:
                    u[nx, ny] = new_dist
                    status[nx, ny] = CONSIDERED
                    heapq.heappush(heap, (new_dist, (nx, ny)))

    return u

size = 100
u = solve_eikonal(size)

plt.imshow(u, cmap='jet',interpolation='nearest')
plt.colorbar(label='Distance')
plt.title('Equiarrival Curves from All Borders of the Square')
plt.show()

level_set = 20

# Binary Image
binary_image = np.isclose(u, level_set, atol=0.5)
plt.imshow(binary_image, cmap='Greys', interpolation='nearest')
plt.title(f'Level set at {level_set}')
plt.show()